In [1]:
import numpy as np
import pandas as pd
import json
import pymongo
import re
from itertools import cycle
import glob
import os
from more_itertools import split_before
from importlib import reload
import retrosheetParser
pd.set_option("display.max_rows", None)

In [32]:
#reload(retrosheetParser)

<module 'retrosheetParser' from 'C:\\Users\\Gregory\\Documents\\GitHub\\retrosheet\\retrosheetParser.py'>

In [2]:
## Locate the retrosheet event files:
dirname = "C:/Users/Gregory/Documents/Baseball2019Events/"
eventfiles = [f for f in os.listdir(dirname) if re.search("\.EV(N|A)$",f)]

In [3]:
# Read the retrosheet event files into python:
events = []
for filename in eventfiles:
    with open(dirname + filename) as f:
        content = f.readlines()
        events = events + content
events = [line.strip('\n') for line in events]

len(events)

407693

In [4]:
# Split the event files by game
content = list(split_before(events, lambda x: x.startswith("id,")))

In [5]:
# Parse out each of the games:
d = [retrosheetParser.RetrosheetEventFormatter(game) for game in content]

In [6]:
# Combine the 'play' DataFrames:
Plays2019 = retrosheetParser.combineGames(d)
# Add attribution to the combined plays DataFrame:
EnhancedPlays2019 = retrosheetParser.enhancePlays(Plays2019)

In [7]:
EnhancedPlays2019.head(10)

Type Inning Team  PlayerID Count PitchSequence  \
Game         PlayNum                                                   
ANA201904040 0        play      1    0  choos001    22         FBBCH   
             1        play      1    0  odorr001    12          BFSS   
             2        play      1    0  andre001    12          CCBX   
             3        play      1    0  gallj002    00             X   
             4        play      1    0  cabra002    30          BBBB   
             5        play      1    0  guzmr001    12          SSBX   
             6        play      1    0  forsl001    12          BCSS   
             7        play      1    0  kinei001    01            CX   
             8        play      1    0  deshd002    12        CB11FT   
             9        play      1    1  calhk001    31         BFBBX   

                                Play  \
Game         PlayNum                   
ANA201904040 0                    HP   
             1                     K   
             2              S9/L.1-2   
             3        HR/8/F.2-H;1-H   
             4                     W   
             5                HR.1-H   
             6                     K   
             7                  S7/G   
             8                     K   
             9               HR/89/F   

                                                                Comment  \
Game         PlayNum                                                      
ANA201904040 0                                                            
             1                                                            
             2                                           On-field Delay   
             3                                                            
             4                                              Mound Visit   
             5        Umpire review of ball hit by Guzman and ruled ...   
             6                                                            
             7                                                            
             8                                                            
             9                                                            

                                                              playSplit  \
Game         PlayNum                                                      
ANA201904040 0                  {'event': 'HP', 'mods': [], 'advs': []}   
             1                   {'event': 'K', 'mods': [], 'advs': []}   
             2          {'event': 'S9', 'mods': ['L'], 'advs': ['1-2']}   
             3        {'event': 'HR', 'mods': ['8', 'F'], 'advs': ['...   
             4                   {'event': 'W', 'mods': [], 'advs': []}   
             5             {'event': 'HR', 'mods': [], 'advs': ['1-H']}   
             6                   {'event': 'K', 'mods': [], 'advs': []}   
             7               {'event': 'S7', 'mods': ['G'], 'advs': []}   
             8                   {'event': 'K', 'mods': [], 'advs': []}   
             9         {'event': 'HR', 'mods': ['89', 'F'], 'advs': []}   

                     subevents  \
Game         PlayNum             
ANA201904040 0            [HP]   
             1             [K]   
             2            [S9]   
             3            [HR]   
             4             [W]   
             5            [HR]   
             6             [K]   
             7            [S7]   
             8             [K]   
             9            [HR]   

                                                         subeventParsed  \
Game         PlayNum                                                      
ANA201904040 0        [{'basicPlay': 'Hit by Pitch', 'playersOut': [...   
             1        [{'basicPlay': 'Strikeout', 'playersOut': ['B'...   
             2        [{'basicPlay': 'Hit - Single', 'playersOut': [...   
             3        [{'basicPlay': 'Hit - Home Run', 'playersOut':...   
             4 

In [8]:
# Troubleshoot innings that should have 3 outs, but don't
(
    EnhancedPlays2019
    .assign(OutsInInning = lambda x: x.groupby(['Game','Inning','Team'])['outsOnPlay'].transform(sum))
    .assign(Inning = lambda x: x.Inning.transform(int))
    .query('OutsInInning != 3')
    .query("Inning < 9")
    .head(100)
    .loc[:,['Inning','Team','Play','Comment','playersOut','outsOnPlay','OutsInInning']]
)

Inning Team                               Play  \
Game         PlayNum                                                   
BAL201906250 27            3    1                               53/G   
             28            3    1                                  K   
             29            3    1                                  W   
             30            3    1           D9/G+.1-H;BX3(E9)(95/TH)   
BOS201904130 62            7    0                               S9/G   
             63            7    0                        PO1(E1).1-2   
             64            7    0                                  W   
             65            7    0                                  K   
             66            7    0               E3/G.2-H(NR)(UR);1-3   
             67            7    0  FC5/G.3-H(RBI)(UR);1X2(54)(E5/TH)   
             68            7    0                                8/L   
NYA201904120 66            7    0                                 NP   
             67            7    0                                7/F   
             68            7    0                             HR/8/F   
             69            7    0                             HR/9/L   
NYA201905050 71            8    1                                 NP   
             72            8    1                                  W   
             73            8    1                             PB.1-2   
             74            8    1                             WP.2-3   
NYN201908220 58            8    1                                 NP   
             59            8    1                                 NP   
             60            8    1                                  W   
             61            8    1                          S9/G+.1-3   
OAK201905090 33            5    0               S2/BG.BX3(95)(E2/TH)   
             34            5    0                               63/G   
             35            5    0                                  K   

                                                                Comment  \
Game         PlayNum                                                      
BAL201906250 27                                                           
             28                                                           
             29                                                           
             30                                                           
BOS201904130 62                                                           
             63                                                           
             64                                                           
             65                                                           
             66                                          On-field Delay   
             67       play originally scored as fielder's choice wit...   
             68                                                           
NYA201904120 66                                                           
             67                                                           
             68                                                           
             69                                    Game called for rain   
NYA201905050 71                                          On-field Delay   
             72                                                           
             73                                                           
             74                                    Game called for rain   
NYN201908220 58                                                           
             59                                                           
             60                                                           
             61                                    Game called for rain   
OAK201905090 33                                                           
             34                                        

In [9]:
# Plays with more than 1 subevent
(
    EnhancedPlays2019
    .assign(nSubevents = lambda x: x.subevents.transform(len))
    .query("nSubevents > 1",engine='python').head(10)
)

Type Inning Team  PlayerID Count PitchSequence  \
Game         PlayNum                                                   
ANA201904040 14       play      1    1  lastt001    01            CX   
             59       play      6    0  cabra002    22        CBFBFX   
             75       play      7    1  lucrj001    12          CFBX   
             95       play      9    1  bourp001    22        BCSBFS   
ANA201904050 3        play      1    0  mazan001    02           CSS   
             9        play      2    0  guzmr001    12         BCCFX   
             23       play      3    1  troum001    21          BCBX   
ANA201904060 33       play      4    0  mazan001    22  SF1*BFB1FFFX   
             79       play      8    1  troum001    32     CB1B11FBS   
ANA201904070 38       play      4    0  cabra002    00             X   

                            Play         Comment  \
Game         PlayNum                               
ANA201904040 14       36(1)1/GDP                   
             59       36(1)1/GDP                   
             75       64(1)3/GDP                   
             95         K+PB.B-1                   
ANA201904050 3             K+SB2                   
             9        36(1)3/GDP                   
             23        4(1)3/GDP                   
ANA201904060 33       36(1)3/GDP                   
             79            K+SB2  On-field Delay   
ANA201904070 38       46(1)3/GDP                   

                                                             playSplit  \
Game         PlayNum                                                     
ANA201904040 14       {'event': '36(1)1', 'mods': ['GDP'], 'advs': []}   
             59       {'event': '36(1)1', 'mods': ['GDP'], 'advs': []}   
             75       {'event': '64(1)3', 'mods': ['GDP'], 'advs': []}   
             95         {'event': 'K+PB', 'mods': [], 'advs': ['B-1']}   
ANA201904050 3              {'event': 'K+SB2', 'mods': [], 'advs': []}   
             9        {'event': '36(1)3', 'mods': ['GDP'], 'advs': []}   
             23        {'event': '4(1)3', 'mods': ['GDP'], 'advs': []}   
ANA201904060 33       {'event': '36(1)3', 'mods': ['GDP'], 'advs': []}   
             79             {'event': 'K+SB2', 'mods': [], 'advs': []}   
ANA201904070 38       {'event': '46(1)3', 'mods': ['GDP'], 'advs': []}   

                       subevents  \
Game         PlayNum               
ANA201904040 14       [36(1), 1]   
             59       [36(1), 1]   
             75       [64(1), 3]   
             95          [K, PB]   
ANA201904050 3          [K, SB2]   
             9        [36(1), 3]   
             23        [4(1), 3]   
ANA201904060 33       [36(1), 3]   
             79         [K, SB2]   
ANA201904070 38       [46(1), 3]   

                                                         subeventParsed  \
Game         PlayNum                                                      
ANA201904040 14       [{'basicPlay': 'Out', 'playersOut': ['1'], 'im...   
             59       [{'basicPlay': 'Out', 'playersOut': ['1'], 'im...   
             75       [{'basicPlay': 'Out', 'playersOut': ['1'], 'im...   
             95       [{'basicPlay': 'Strikeout', 'playersOut': ['B'...   
ANA201904050 3        [{'basicPlay': 'Strikeout', 'playersOut': ['B'...   
             9        [{'basicPlay': 'Out', 'playersOut': ['1'], 'im...   
             23       [{'basicPlay': 'Out', 'playersOut': ['1'], 'im...   
ANA201904060 33       [{'basicPlay': 'Out', 'playersOut': ['1'], 'im...   
             79       [{'basicPlay': 'Strikeout', 'playersOut': ['B'...   
ANA201904070 38       [{'basicPlay': 'Out', 'playersOut': ['1'], 'im...   

                     basicPlayDesc             fullPlayDesc playersOut  \
Game         PlayNum                                                     
ANA201904040 14        Double Play              Double Play     [1, B]   
             59        Double Play              Double Play     [1, B]   
             

In [13]:
#event_collection = db['events']

In [22]:
#event_collection.insert_one(d)

In [123]:
#event_collection.find_one({"id":"BOS201904090"})